### Docling

#### Docling 기본 활용법

Docling은 Layout 분석, 테이블 분석, 텍스트 인식 등 딥러닝 모델을 활용하는 모듈이 많습니다.

따라서 GPU가 없는 환경에서 테스트하는 경우, 셀의 실행 시간이 길 수 있습니다.

강의에서 진행하는 실습 환경은 RAM: 32GB / GPU: RTX 4060 기준으로 진행되었습니다.

테스트가 목적인 경우 Colab에서 실행하시기를 추천드립니다.

In [ ]:
# %pip install -U docling

In [ ]:
from docling.document_converter import DocumentConverter
source = "data/국가별 공공부문 AI 도입 및 활용 전략.pdf"  # document per local path or URL
converter = DocumentConverter()
result = converter.convert(source)

Docling의 Converter 결과물은 markdown, html, doctag(docling의 XML 기반 태그)로 출력할 수 있습니다.

In [ ]:
from IPython.display import Markdown

# 마크다운 형식으로 문서를 내보내기
# export_to_markdown() 메서드는 문서를 마크다운 형식으로 변환하여 반환합니다.
display(Markdown(result.document.export_to_markdown()))

In [ ]:
# HTML 형식으로 문서를 내보내기
# export_to_html() 메서드는 문서를 HTML 형식으로 변환하여 반환합니다.
result.document.export_to_html()

In [ ]:
# doctag 형식으로 문서를 내보내기
# export_to_doctag() 메서드는 문서를 doctag 형식으로 변환하여 반환합니다.
print(result.document.export_to_doctags())

#### Docling의 Custom pipeline 알아보기

In [8]:
# JSON 모듈 가져오기 - 데이터 직렬화 및 역직렬화에 사용
import json
# 로깅 모듈 가져오기 - 디버깅 및 정보 기록에 사용
import logging
# 시간 측정 모듈 가져오기 - 실행 시간 측정에 사용
import time
# 파일 경로 처리 모듈 가져오기 - 파일 및 디렉토리 경로 관리에 사용
from pathlib import Path
# 로거 인스턴스 생성 - 현재 모듈의 로깅을 위해 사용
_log = logging.getLogger(__name__)

Docling은 pipeline option을 통해 문서의 OCR 처리 여부나 Table 구조 인식, 셀 매칭과 같은 작업을 쉽게 수행할 수 있습니다.
- OCR은 EasyOCR로 구동됩니다.
- Table 구조 인식은 Docling에서 개발한 Tableformer 모델을 통해 구동됩니다.

In [6]:
# 기본 데이터 모델 가져오기 - 입력 형식 정의에 사용
from docling.datamodel.base_models import InputFormat
# PDF 파이프라인 옵션 가져오기 - PDF 처리 설정에 사용
from docling.datamodel.pipeline_options import (
    PdfPipelineOptions,
)
# 문서 변환기 및 PDF 형식 옵션 가져오기 - 문서 변환에 사용
from docling.document_converter import DocumentConverter, PdfFormatOption

# PDF 파이프라인 옵션 설정
pipeline_options = PdfPipelineOptions()

pipeline_options.do_ocr = False  # OCR 기능 비활성화 (이미 텍스트가 있는 PDF 사용)
pipeline_options.ocr_options.lang = ["ko"]  # OCR 언어를 한국어로 설정 (OCR 사용 시)

pipeline_options.do_table_structure = True  # 표 구조 인식 활성화 --> docling의 tableformer 활용해 표 상세 사항 파악
pipeline_options.table_structure_options.do_cell_matching = True  # 표 셀 매칭 활성화

# 문서 변환기 생성 및 PDF 형식 옵션 설정
doc_converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

In [ ]:
# 입력 문서 경로 설정
input_doc_path = "data/국가별 공공부문 AI 도입 및 활용 전략.pdf"
# 변환 시작 시간 기록
start_time = time.time()
# 문서 변환 실행
conv_result = doc_converter.convert(input_doc_path)
# 변환 종료 시간 계산
end_time = time.time() - start_time

# 변환 완료 시간 로깅
_log.info(f"Document converted in {end_time:.2f} seconds.")

## 결과 내보내기
# 출력 디렉토리 설정
output_dir = Path("scratch")
# 출력 디렉토리가 없으면 생성
output_dir.mkdir(parents=True, exist_ok=True)
# 파일 이름 추출 (확장자 제외)
doc_filename = conv_result.input.file.stem

# 문서 JSON 형식으로 내보내기:
with (output_dir / f"{doc_filename}.json").open("w", encoding="utf-8") as fp:
    fp.write(json.dumps(conv_result.document.export_to_dict()))

# 텍스트 형식으로 내보내기:
with (output_dir / f"{doc_filename}.txt").open("w", encoding="utf-8") as fp:
    fp.write(conv_result.document.export_to_text())

# 마크다운 형식으로 내보내기:
with (output_dir / f"{doc_filename}.md").open("w", encoding="utf-8") as fp:
    fp.write(conv_result.document.export_to_markdown())

# doctag 형식으로 내보내기:
with (output_dir / f"{doc_filename}.doctags").open("w", encoding="utf-8") as fp:
    fp.write(conv_result.document.export_to_doctags())

#### Docling의 OCR, 테이블 구조 인식 비교해보기

`do_table_structure` 파라미터는 Docling의 Tableformer 모델로 표를 더 정확히 인식할지 선택하는 옵션입니다.
이를 활용하면, 기존 위치기반 테이블 내 텍스트 추출을 넘어서 테이블 내의 정보를 더 정갈하게 출력할 수 있습니다.

In [ ]:
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.datamodel.base_models import InputFormat
from docling.document_converter import DocumentConverter, PdfFormatOption

def convert_with_table_option(pdf_path, enable_table):
    opts = PdfPipelineOptions()
    opts.do_ocr = False
    opts.do_table_structure = enable_table
    opts.table_structure_options.do_cell_matching = True

    converter = DocumentConverter(
        format_options={InputFormat.PDF: PdfFormatOption(pipeline_options=opts)}
    )
    return converter.convert(pdf_path)

# 표 추출 꺼짐
res_without_table = convert_with_table_option(input_doc_path, enable_table=False)
print(f"❌ Table disabled: tables found = {len(res_without_table.document.tables)}")

# 표 추출 켜짐
res_with_table = convert_with_table_option(input_doc_path, enable_table=True)
print(f"✅ Table enabled: tables found = {len(res_with_table.document.tables)}")


In [ ]:
res_with_table.document.pictures

OCR 기능을 활용하는 것만이 정답은 아닙니다. 텍스트가 디지털로 인식되는(드래그 가능한) PDF의 경우 OCR을 사용하지 않는 것이 추출에 더 좋습니다.
OCR은 스캔된 문서의 경우 유용한 선택지이며, 기본 옵션은 EasyOCR입니다. 언어를 ko로 설정하면 한글 OCR에도 활용할 수 있습니다.

In [ ]:
def convert_with_ocr_option(pdf_path, enable_ocr):
    opts = PdfPipelineOptions()
    opts.do_ocr = enable_ocr
    opts.ocr_options.lang = ["ko"]
    opts.do_table_structure = False  # 표 관련 영향 없게 끔

    converter = DocumentConverter(
        format_options={InputFormat.PDF: PdfFormatOption(pipeline_options=opts)}
    )
    return converter.convert(pdf_path)

# OCR 꺼짐
res_without_ocr = convert_with_ocr_option(input_doc_path, enable_ocr=False)
print(f"❌ OCR disabled: text blocks = {len(res_without_ocr.document.texts)}")

# OCR 켜짐
res_with_ocr = convert_with_ocr_option(input_doc_path, enable_ocr=True)
print(f"✅ OCR enabled: text blocks = {len(res_with_ocr.document.texts)}")


OCR로 텍스트를 추출한 결과, Text block이 더 많은 이유는 단순히 텍스트를 더 잘게 쪼개서 인식했기 때문입니다.
OCR은 텍스트 영역에 대해 텍스트 인식 모델을 구동한 결과를 저장하는 것이기 때문에, 그 영역이 디지털 텍스트로 인식되는 것과 달리 중간에 쪼개질 수 있습니다.

In [ ]:
def preview_ocr_comparison(res_with_ocr, res_without_ocr, num_chars=1000):
    text_with_ocr = res_with_ocr.document.export_to_text()
    text_without_ocr = res_without_ocr.document.export_to_text()

    print("="*30)
    print("🟢 WITH OCR:")
    print("="*30)
    print(text_with_ocr[:num_chars])
    print("\n\n")
    print("="*30)
    print("🔴 WITHOUT OCR:")
    print("="*30)
    print(text_without_ocr[:num_chars])

    print("\n\n")
    print("="*30)
    print("📊 SUMMARY:")
    print("="*30)
    print(f"- OCR 켠 텍스트 길이: {len(text_with_ocr):,} characters")
    print(f"- OCR 끈 텍스트 길이: {len(text_without_ocr):,} characters")
    
    if len(text_with_ocr) > len(text_without_ocr):
        print("✅ OCR 결과가 더 많은 텍스트를 추출했음 (아마 스캔 PDF거나 이미지 기반 문서)")
    elif len(text_with_ocr) < len(text_without_ocr):
        print("⚠️ OCR 결과가 오히려 줄었음... OCR 엔진이 대충 찍었거나 원래 텍스트 레이어가 있었던 듯")
    else:
        print("🤔 두 결과가 비슷함. OCR 옵션이 큰 영향을 주지 않았을 수도 있음.")

# 예시 호출
preview_ocr_comparison(res_with_ocr, res_without_ocr, num_chars=10000)


In [ ]:
def debug_text_output(result, name="result"):
    text = result.document.export_to_text()
    print(f"\n📄 {name} text length: {len(text)} characters")
    print(f"Preview (first 300 chars):\n{text[:300]!r}")

    lines = text.splitlines()
    print(f"🧱 splitlines() count: {len(lines)}")
    for i, line in enumerate(lines[:100]):
        print(f"  {i:02d}: {line}")

# 실행 예시
debug_text_output(res_with_ocr, name="OCR ON")
debug_text_output(res_without_ocr, name="OCR OFF")


#### 스캔 문서 OCR 해보기

In [ ]:
def convert_with_ocr_option(pdf_path, enable_ocr):
    opts = PdfPipelineOptions()
    opts.do_ocr = enable_ocr
    opts.ocr_options.lang = ["ko"]
    opts.do_table_structure = True  # 표 관련 영향 없게 끔
    opts.table_structure_options.do_cell_matching = True
    
    converter = DocumentConverter(
        format_options={InputFormat.PDF: PdfFormatOption(pipeline_options=opts)}
    )
    return converter.convert(pdf_path)

# OCR 꺼짐
res_without_ocr = convert_with_ocr_option("data/은행 확인서.pdf", enable_ocr=False)
print(f"❌ OCR disabled: text blocks = {len(res_without_ocr.document.texts)}")

# OCR 켜짐
res_with_ocr = convert_with_ocr_option("data/은행 확인서.pdf", enable_ocr=True)
print(f"✅ OCR enabled: text blocks = {len(res_with_ocr.document.texts)}")

In [ ]:
print(res_with_ocr.document.export_to_markdown())

#### Docling으로 이미지까지 처리하기

Docling의 picture_description_options 옵션을 활용하면, 페이지에서 추출된 이미지에 대한 캡션과 주석을 생성할 수 있습니다.

OpenAI나 Claude와 같이 Vision을 지원하는 API를 활용할 수 있습니다.

In [17]:
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import (
    PdfPipelineOptions,
    PictureDescriptionApiOptions,
)
from docling.document_converter import DocumentConverter, PdfFormatOption
import os

openai_api_key = os.getenv("OPENAI_API_KEY")

# OpenAI/ChatGPT Vision API를 사용하는 옵션
openai_vlm_options = PictureDescriptionApiOptions(
    url="https://api.openai.com/v1/chat/completions",  # 예시 URL
    params=dict(
        model="gpt-4.1",
    ),
    headers={"Authorization": f"Bearer {openai_api_key}"},  # 실제 키로 대체할 것
    prompt="Describe this image in two sentences. Focus on factual accuracy.",
    timeout=30,
)

pipeline_options = PdfPipelineOptions()
pipeline_options.do_picture_description = True
pipeline_options.picture_description_options = openai_vlm_options
pipeline_options.images_scale = 2.0
pipeline_options.generate_picture_images = True
pipeline_options.enable_remote_services=True

converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

doc = converter.convert("data/국가별 공공부문 AI 도입 및 활용 전략.pdf").document


In [ ]:
from docling_core.types.doc import PictureItem

for element, _level in doc.iterate_items():
    if isinstance(element, PictureItem):
        print(
            f"Picture {element.self_ref}\n"
            f"Caption: {element.caption_text(doc=doc)}\n"
            f"Annotations: {element.annotations}"
        )

로컬 모델을 통해 이미지 주석을 생성할 때는 Ollama API를 활용하거나 SmolVLM과 granite-vision을 활용할 수 있습니다.

*로컬 실행의 경우 CUDA의 Flash attention이 지원되어야 합니다. 해당 모듈은 윈도우에서 타 라이브러리와의 의존성 문제가 심하여 강의 실습에서는 진행하지 않습니다.

*이를 테스트하고자 할 경우, Colab 환경에서 GPU 옵션을 키고 실행하시기를 권장드립니다.

In [ ]:
from pathlib import Path
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import (
    PdfPipelineOptions,
    smolvlm_picture_description,
)
from docling.document_converter import DocumentConverter, PdfFormatOption

import os
os.environ["HF_HUB_ENABLE_HARDLINKS"] = "false"


# SmolDocling 설정 복사해서 prompt만 맞춤
smol_options = smolvlm_picture_description
smol_options.prompt = "Briefly explain what is shown in this image."

pipeline_options = PdfPipelineOptions()
pipeline_options.do_picture_description = True
pipeline_options.picture_description_options = smol_options
pipeline_options.images_scale = 2.0
pipeline_options.generate_picture_images = True

converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
    }
)

doc = converter.convert("data/국가별 공공부문 AI 도입 및 활용 전략.pdf").document


그런데, 모든 이미지를 VLM으로 처리하면 지연 시간이 너무 길어집니다.

따라서, 텍스트가 많이 포함된 정보량이 풍부한 이미지에 대해서만 VLM 추론을 실행하는 것이 효과적입니다.

이를 위해, Docling으로 추출한 이미지를 폴더에 저장하고, 해당 폴더 내 여러 이미지 중 OCR 결과 텍스트가 많이 포함된 이미지만 처리할 수도 있습니다.

In [19]:
from docling.datamodel.base_models import InputFormat
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.pipeline_options import (
    PdfPipelineOptions,
    )

# Configure pipeline options
pipeline_options = PdfPipelineOptions()
pipeline_options.generate_picture_images = True
pipeline_options.generate_page_images = True
pipeline_options.do_table_structure = True
pipeline_options.images_scale = 2.0  # Adjust the scale as needed

# Set format options
format_options = {
    InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)
}

# Initialize the converter with format options
converter = DocumentConverter(
    format_options=format_options)

# Convert the document
result = converter.convert("data/국가별 공공부문 AI 도입 및 활용 전략.pdf")


Docling에서 convert 결과로 내놓는 document에서 pictures 속성에 접근하면 이미지 정보를 볼 수 있습니다.

아래 코드는 각 이미지 정보를 순회하면서 이미지를 숫자로 표현한 base64 정보를 추출하고

페이지 넘버와 함께 하나의 폴더에 저장하는 코드입니다.

In [ ]:
import os
import base64
from PIL import Image
from io import BytesIO

output_dir = "extracted_images"
os.makedirs(output_dir, exist_ok=True)

pictures = result.document.pictures

for idx, picture in enumerate(pictures):
    try:
        # Url 객체를 문자열로 변환 후 base64 부분 추출
        uri_str = str(picture.image.uri)
        base64_data = uri_str.split(',')[1]

        # 디코딩 및 PIL 변환
        image_data = base64.b64decode(base64_data)
        image = Image.open(BytesIO(image_data))

        # 페이지 번호 기반 파일 이름
        page_no = picture.prov[0].page_no if picture.prov else 0
        filename = f"page_{page_no}_img_{idx+1}.png"

        image.save(os.path.join(output_dir, filename))
        print(f"✅ Saved: {filename}")
    except Exception as e:
        print(f"❌ Failed to save image {idx}: {e}")


이제 EasyOCR을 활용해 이미지 속 텍스트의 수를 기준으로 정보량이 풍부한 이미지와 그렇지 않은 이미지를 구분할 수 있습니다.

그 이후는 이전 예시와 마찬가지로 VLM 추론을 통해 이미지 캡션을 수행하면 됩니다.

In [ ]:
import os
from PIL import Image
import numpy as np
import easyocr

# EasyOCR reader
reader = easyocr.Reader(['ko', 'en'], gpu=True)

# 이미지 폴더 경로
image_folder = "extracted_images"

# 결과 저장용 리스트
rich_images = []   # 정보 풍부
poor_images = []   # 정보 부족

# 폴더 내 이미지 순회
for filename in os.listdir(image_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(image_folder, filename)

        try:
            # 이미지 불러오기
            img = Image.open(img_path).convert("RGB")
            img_np = np.array(img)

            # OCR 수행
            results = reader.readtext(img_np)

            texts = [res[1] for res in results]
            total_text = ''.join(texts)

            # 조건 검사
            if len(texts) >= 3 and len(total_text) >= 30:
                rich_images.append((filename, len(texts), len(total_text)))
            else:
                poor_images.append((filename, len(texts), len(total_text)))

        except Exception as e:
            print(f"❌ 오류 발생 - {filename}: {e}")

# 결과 출력
print("\n📘 정보량 많은 이미지:")
for name, blocks, chars in rich_images:
    print(f"✔ {name} - 블럭 {blocks}개, 총 글자수 {chars}")

print("\n📄 정보 부족한 이미지:")
for name, blocks, chars in poor_images:
    print(f"✖ {name} - 블럭 {blocks}개, 총 글자수 {chars}")